# Homework 6 (Extra Credit)
In this assignment, you will practice using Pandas to solve problems you implemented with base Python. For those of you who attempted Part 6 of Homework 2/3, it is exactly the same as what you did then. This time, however, you will use the power of Pandas to generate your answers.

#### Part 1 - Describe my students

In the cell below, I've written a bit of code to load some data for you to analyze.

`student_info` contains personal information for several students. You will use this to answer all of the questions in Part 4.

In [2]:
import pandas as pd
student_info = pd.read_csv('students.csv')

In [3]:
student_info.head()

,student_id,first,last,gender,class,major,gpa
0,5a397209-3782-4764-a285-10fae807ee71,Janis,Brown,Female,Junior,Economics,3.12
1,e26c3d69-3c74-49b6-81d7-47232787fad9,Timothy,Bishop,Male,Sophomore,Economics,3.48
2,975c1581-5ba2-430c-a3d1-01ce03bd83f9,Elizabeth,Owens,Female,Freshman,Finance,3.40
3,6081f91d-365c-46ce-ad1b-38af120781d9,Edward,Pearson,Male,Freshman,Math,3.84
4,84cec8f4-0b64-44ce-a628-c0eb73f6ca6f,Lisa,Gonzalez,Female,Junior,Finance,4.00


1.1) How many different majors are represented by these students?

In [4]:
len(set(student_info['major']))

6

In [7]:
#You could also you Pandas built-in "nunique" method
student_info['major'].nunique()

6

1.2) What is the most popular major amongst each gender?

#### Approach 1

In [13]:
def find_most_common_thing(df, column):
    counts = df.groupby(column).size()
    largest_index = counts.idxmax()
    return largest_index, counts[largest_index]

In [14]:
find_most_common_thing(student_info, 'major')

('Finance', 2241)

In [16]:
for gender in set(student_info['gender']):
    gender_df = student_info[student_info['gender']==gender]
    most_popular = find_most_common_thing(gender_df, 'major')
    print(gender, most_popular[0], most_popular[1])

Male Engineering 1524
Female Economics 1500


#### Approach 2

In [18]:
gender_major_counts = student_info.groupby(['gender', 'major']).size()

In [21]:
for gender in set(student_info['gender']):
    gender_count = gender_major_counts[gender]
    largest_index = gender_count.idxmax()
    largest_value = gender_count[largest_index]
    print(gender, largest_index, largest_value)

Male Engineering 1524
Female Economics 1500


1.3) For each major, what is the ratio of males to females in each class? For example, in the major or Economics, what are the ratios of males to females in the freshman class, sophomore class, junior class, and senior class? (NOTE: I *strongly* recommend creating a function to help you answer this one).

In [31]:
genders = set(student_info['gender'])
majors = set(student_info['major'])
classes = set(student_info['class'])

In [35]:
gender_counts = student_info.groupby(['major', 'class', 'gender']).size()
gender_counts.head()

major      class     gender
Chemistry  Freshman  Female    243
                     Male      125
           Junior    Female    245
                     Male      130
           Senior    Female    224
dtype: int64

In [36]:
def compute_ratio(major, class_):
    num_males = gender_counts.loc[(major, class_, 'Male')]
    num_females = gender_counts.loc[(major, class_, 'Female')]
    ratio = num_males / num_females
    return ratio

In [40]:
all_ratios = []
for major in majors:
    for major_class in classes:
        ratio = compute_ratio(major, major_class)
        all_ratios.append([major, major_class, ratio])
all_ratios_df = pd.DataFrame(all_ratios, columns=['major', 'class', 'm/f_ratio'])

In [42]:
all_ratios_df.sort_values(['major', 'class'])

,major,class,m/f_ratio
7,Chemistry,Freshman,0.514403
5,Chemistry,Junior,0.530612
6,Chemistry,Senior,0.558036
4,Chemistry,Sophomore,0.596154
23,Economics,Freshman,0.281977
21,Economics,Junior,0.298295
22,Economics,Senior,0.352113
20,Economics,Sophomore,0.320106
11,Engineering,Freshman,3.571429
9,Engineering,Junior,2.524194


1.4) Which major has the highest average gpa for men and which major has the highest gpa for women?

In [49]:
gender_major_gpas = student_info.groupby(['gender', 'major'])['gpa'].mean()
gender_major_gpas.head()

gender  major      
Female  Chemistry      3.395772
        Economics      3.495707
        Engineering    2.995098
        Finance        3.654529
        Math           3.268426
Name: gpa, dtype: float64

In [54]:
def get_max_gpa_major(series):
    max_major = series.idxmax()
    max_gpa = series[max_major]
    return max_major, max_gpa

In [58]:
for gender in genders:
    print(gender, get_max_gpa_major(gender_major_gpas[gender]))

Male ('Finance', 3.5685354025218197)
Female ('Finance', 3.6545289256198346)
